In [9]:
import torch
smooth_scales = torch.load("act_scales/llama2-7b-hf-smooth.pt")
act_scales = torch.load("act_scales/llama2-7b-hf-static.pt")
act_w_scales = torch.load("act_scales/llama2-7b-hf-static-weight.pt")
smooth_act_scales = torch.load("act_scales/llama2-7b-hf-smooth-static.pt")

In [10]:
for key in act_scales.keys():
    # print(key)
    # print(smooth_scales[key])
    # print(act_scales[key]['input'].max().item(), act_scales[key]['input'].min().item())
    # print(act_w_scales[key]['input'].max().item(), act_scales[key]['input'].min().item())
    # print(smooth_act_scales[key]['input'])
    # print(key, act_scales[key]['input'].max().item(), act_scales[key]['output'].max().item())
    print(key, act_w_scales[key]['weight'].max().item(), act_w_scales[key]['weight'].min().item())

model.layers.0.self_attn.q_proj 0.7734375 0.02099609375
model.layers.0.self_attn.k_proj 0.81640625 0.0224609375
model.layers.0.self_attn.v_proj 0.134765625 0.0223388671875
model.layers.0.self_attn.o_proj 0.451171875 0.0281982421875
model.layers.0.mlp.gate_proj 0.890625 0.0380859375
model.layers.0.mlp.up_proj 0.341796875 0.038818359375
model.layers.0.mlp.down_proj 0.52734375 0.055908203125
model.layers.1.self_attn.q_proj 0.47265625 0.0242919921875
model.layers.1.self_attn.k_proj 0.57421875 0.0302734375
model.layers.1.self_attn.v_proj 0.1279296875 0.0147705078125
model.layers.1.self_attn.o_proj 0.58984375 0.031494140625
model.layers.1.mlp.gate_proj 1.09375 0.052978515625
model.layers.1.mlp.up_proj 0.40625 0.04248046875
model.layers.1.mlp.down_proj 1.5625 0.05859375
model.layers.2.self_attn.q_proj 1.1015625 0.0400390625
model.layers.2.self_attn.k_proj 0.373046875 0.0277099609375
model.layers.2.self_attn.v_proj 0.1474609375 0.031494140625
model.layers.2.self_attn.o_proj 0.60546875 0.043212

In [21]:
projections = ["q", "k", "v", "o"]
mlps = ["gate", "up", "down"]
header_projections = [f"{proj}_input, {proj}_output" for proj in projections]
header_mlps = [f"{mlp}_input, {mlp}_output" for mlp in mlps]
csv_header = f"Layer, " + ", ".join(header_projections) + ", " + ", ".join(header_mlps) + "\n"
csv_lines = [csv_header]

# 收集每層的數據
for i in range(32):  # 假設有 32 層
    layer_data = [f"Layer {i+1}"]
    for proj in projections:
        input_key = f"model.layers.{i}.self_attn.{proj}_proj"
        output_key = f"model.layers.{i}.self_attn.{proj}_proj"
        input_max = act_scales.get(input_key, {}).get("input", torch.tensor([float('nan')])).max().item()
        output_max = act_scales.get(output_key, {}).get("output", torch.tensor([float('nan')])).max().item()
        layer_data.append(f"{input_max}")
        layer_data.append(f"{output_max}")
    for mlp in mlps:
        input_key = f"model.layers.{i}.mlp.{mlp}_proj"
        output_key = f"model.layers.{i}.mlp.{mlp}_proj"
        input_max = act_scales.get(input_key, {}).get("input", torch.tensor([float('nan')])).max().item()
        output_max = act_scales.get(output_key, {}).get("output", torch.tensor([float('nan')])).max().item()
        layer_data.append(f"{input_max}")
        layer_data.append(f"{output_max}")
    csv_lines.append(", ".join(layer_data) + "\n")

# 將 CSV 內容合併成一個字符串
csv_content = "".join(csv_lines)

# 打印 CSV 內容（或保存到文件）
print(csv_content)

# 如果需要將結果保存到文件
file_path = 'layer_act_scales.csv'
with open(file_path, 'w') as file:
    file.write(csv_content)

file_path

Layer, q_input, q_output, k_input, k_output, v_input, v_output, o_input, o_output, gate_input, gate_output, up_input, up_output, down_input, down_output
Layer 1, 4.97265625, 12.65625, 4.97265625, 6.12109375, 4.97265625, 1.2568359375, 1.2568359375, 0.93701171875, 2.275390625, 3.50390625, 2.275390625, 1.7900390625, 2.435546875, 2.052734375
Layer 2, 9.5390625, 11.015625, 9.5390625, 8.546875, 9.5390625, 1.806640625, 1.2919921875, 2.318359375, 1.490234375, 44.0, 1.490234375, 38.0625, 1675.0, 2718.0
Layer 3, 8.765625, 11.234375, 8.765625, 14.0859375, 8.765625, 3.646484375, 1.2890625, 3.099609375, 5.6875, 5.015625, 5.6875, 2.880859375, 4.625, 4.296875
Layer 4, 10.03125, 14.0859375, 10.03125, 16.453125, 10.03125, 3.66796875, 2.751953125, 4.37890625, 3.4765625, 6.984375, 3.4765625, 3.20703125, 10.9765625, 6.6875
Layer 5, 10.828125, 12.40625, 10.828125, 18.59375, 10.828125, 4.16015625, 2.87890625, 3.4296875, 7.40234375, 5.51171875, 7.40234375, 3.78125, 11.5390625, 9.421875
Layer 6, 10.9609375, 1

'layer_act_scales.csv'